## Load synthetic InSAR Data and compare to Observations
To be run in environment `seissol`

This notebook is an adaptation of Mathilde's adaptation (`compare_geodqeticMM_mathilde-original.py`) of Thomas's code. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pandas as pd
import os
import shapefile as shp
import rasterio
import seissolxdmf
import time
from scipy import spatial
from multiprocessing import Pool, cpu_count, Manager
from pyproj import Transformer
from cmcrameri import cm
import matplotlib
from rasterio.plot import show
import scipy.interpolate as interp

ps = 12 # set font size
matplotlib.rcParams.update({"font.size": ps})
plt.rcParams["font.family"] = "sans"
matplotlib.rc("xtick", labelsize=ps)
matplotlib.rc("ytick", labelsize=ps)


## Define Functions

In [64]:
ps = 12 # set font size
matplotlib.rcParams.update({"font.size": ps})
plt.rcParams["font.family"] = "sans"
matplotlib.rc("xtick", labelsize=ps)
matplotlib.rc("ytick", labelsize=ps)


def nanrms(x, axis=None):
    return np.sqrt(np.nanmean(x**2, axis=axis))

def setup_map(ax,extentmap,gridlines_left=True, draw_labels=True):
    """Setup the background map with cartopy"""
    ax.set_extent(extentmap, crs=ccrs.PlateCarree())
    scale = "10m"
    ax.add_feature(cfeature.LAND.with_scale(scale), facecolor='whitesmoke', rasterized=True)
    ax.add_feature(cfeature.OCEAN.with_scale(scale), rasterized=True)
    ax.add_feature(cfeature.COASTLINE.with_scale(scale))
    ax.add_feature(cfeature.BORDERS.with_scale(scale), linestyle=":")
    locs = np.arange(-180, 180, 1.0)
    gl = ax.gridlines(draw_labels=draw_labels, ylocs=locs, xlocs=locs)
    gl.right_labels = False
    gl.top_labels = False
    gl.left_labels = gridlines_left
    for fn in [
        "/Users/hyin/agsd/projects/insar/2021_haiti/dynamic-rupture/fault-geometry/combined-drm-geometry_v05/combined-drm-geometry_v05.shp" # shape file with fault traces, each as a line object
    ]:
        sf = shp.Reader(fn)
        for sr in sf.shapeRecords():    # for each fualt?
            listx = []
            listy = []
            for xNew, yNew in sr.shape.points:
                listx.append(xNew)
                listy.append(yNew)
            ax.plot(listx, listy, "k", linewidth=0.5)

def read_observation_data_one_band(fn):
    with rasterio.open(fn) as src:
        ew = src.read(1)
        print("band 1 has shape", ew.shape)
        ds = downsampling
        height, width = ew.shape
        cols, rows = np.meshgrid(np.arange(width), np.arange(height))
        lon_g, lat_g = rasterio.transform.xy(src.transform, rows, cols)
        lon_g = np.array(lon_g)[::ds, ::ds]
        lat_g = np.array(lat_g)[::ds, ::ds]
        ew = ew[::ds, ::ds]
        return lon_g, lat_g, ew


def compute_LOS_displacement_SeisSol_data(
    lon_g, lat_g, theta_g, phi_g, lonlat_barycenter, U, V, W
):

    if band == "azimuth":
        D_los = U * np.sin(phi_g) + V * np.cos(phi_g)
    else:
        D_los = W * np.cos(theta_g) + np.sin(theta_g) * (
            U * -np.cos(phi_g) + V * np.sin(phi_g)
        )
        # D_los = W * np.sin(theta_inter) + np.cos(theta_inter) * (U * np.cos(phi_inter) + V * np.sin(phi_inter))
    return -D_los

def compute_LOS_displacement_SeisSol_data2(lon_g, lat_g, vx, vy, vz, lonlat_barycenter, U,V,W):
    D_los = U * vx + V * vy + W * vz
    return -D_los
    
    
def read_seissol_surface_data(xdmfFilename):
    """
    Read unstructured free surface output and associated data.
    compute cell_barycenter
    
    
    Parameters
    ----------
    xdmfFilename : string
        Full path to the .xdmf file

    Returns
    -------
    lons, lats, lonlat_barycenter, connect, U, V, W
    lons, lats: n x 1 array the locations of each grid point where surface deformation is defined in the model. 
        I believe this is in WGS84 but need to check
        Where n is the number of points in the surface mesh
        i.e. for Haiti low res mesh this is (16347,)
    lonlat_barycenter: n x 2 array where each column is the lon and lat coords of the barycenter of a triangle
        Where n is the number of triangles (need to check this!)
        i.e. for Haiti low res mesh this is (32526, 2)
    connect: n x 3 array, where the columns contain indices for the corners of each triangle at the surface
    U, V, W: each are an nd-array where each is a component of modeled surface deformation:
        U = u1
        V = u2
        W = u3
    """
    sx = seissolxdmf.seissolxdmf(xdmfFilename)

    xyz = sx.ReadGeometry()
    connect = sx.ReadConnect()
    U = sx.ReadData("u1", sx.ndt - 1)
    V = sx.ReadData("u2", sx.ndt - 1)
    W = sx.ReadData("u3", sx.ndt - 1)
    # project the data to geocentric (lat, lon)

    myproj = "+proj=tmerc +datum=WGS84 +k=0.9996 +lon_0=37.0 +lat_0=37.0"
    transformer = Transformer.from_crs(myproj, "epsg:4326", always_xy=True) # WGS84
    lons, lats = transformer.transform(xyz[:, 0], xyz[:, 1])
    xy = np.vstack((lons, lats)).T  # gives a grid of lon,lat values, need to check the format

    # compute triangule barycenter
    lonlat_barycenter = (
        xy[connect[:, 0], :] + xy[connect[:, 1], :] + xy[connect[:, 2], :]
    ) / 3.0

    return lons, lats, lonlat_barycenter, connect, U, V, W

def interpolate_seissol_surf_output(lonlat_barycenter, U, df):
    """interpolate SeisSol free surface output to GPS data location"""

    Fvsm = interp.LinearNDInterpolator(lonlat_barycenter, U)
    locGPS = np.vstack((df["lon"].to_numpy(), df["lat"].to_numpy())).T

    Fvsm = interp.LinearNDInterpolator(lonlat_barycenter, U)
    ui = Fvsm.__call__(locGPS)
    Fvsm = interp.LinearNDInterpolator(lonlat_barycenter, V)
    vi = Fvsm.__call__(locGPS)
    Fvsm = interp.LinearNDInterpolator(lonlat_barycenter, W)
    wi = Fvsm.__call__(locGPS)

    return ui, vi, wi

In [67]:
# Define SeisSol surface XDMF file
xdmfFilename = '/Users/hyin/agsd/projects/insar/2021_haiti/dynamic-rupture/data_tmp/dynamic-rupture/jobid_3422948/output_extracted-surface.xdmf'
# Read Modeled surface deformation data from surface xdmf
lons2, lats2, lonlat_barycenter2, connect2, U2, V2, W2=read_seissol_surface_data(xdmfFilename)


## Read in InSAR Observation data

In [66]:
## Read in InSAR observation data
# Write to geodataframe, with lon / lat fields as geometry
# Set geometry reference frame as WGS84

import geopandas as gpd

filebase = 'A2_A043_20210818-20210901_los'
df = pd.read_csv('data/csv/'+filebase+'.csv')
gdf = gpd.GeoDataFrame(
    geometry=gpd.points_from_xy(df.lon, df.lat, df.elevation, crs="EPSG:4326"), data=df    
)

lon_obs, lat_obs = df.lon.array, df.lat.array

lon_obs

<NumpyExtensionArray>
[  285.8357, 285.840434, 285.845167, 285.849901, 285.854634, 285.859368,
 285.864101, 285.868835, 285.873569, 285.878302,
 ...
 286.484198, 286.488931, 286.493665, 286.498399, 286.503132, 286.507866,
 286.512599, 286.517333, 286.522066,   286.5268]
Length: 19110, dtype: float64

In [68]:
# Create an array with lons and lats of observations locations
locData = np.vstack((lon_obs,lat_obs)).T

In [69]:
## Interpolate modeled data at the barycenter of each triangle
# Interpolate synthetic data to make it continuous 
Fvsm2 = interp.LinearNDInterpolator(lonlat_barycenter2, U2)
# Find the x, y, z components of deformation (ui, vi, wi) at that observation point
ui = Fvsm2.__call__(locData)
Fvsm2 = interp.LinearNDInterpolator(lonlat_barycenter2, V2)
vi = Fvsm2.__call__(locData)
Fvsm2 = interp.LinearNDInterpolator(lonlat_barycenter2, W2)
wi = Fvsm2.__call__(locData)

## Plot initial data

In [ ]:
# Plot figure
vmax = 4.0
vmin = -vmax
if band in ["azimuth", "range"]:
    extentmap=[36.6, 38.4, 36.7, 38.2]
else:
    extentmap=[36, 38.8, 36.0, 38.6]

fig = plt.figure()
fig.set_size_inches(5, 4.5)
ax = []
ax.append(fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree()))
setup_map(ax[0],extentmap)
show(obs_to_plot, transform=src.transform, cmap=cm.vik,vmin=vmin,vmax=vmax)

## Misc
Everything below here was worked on at various times and is now under construction

Let's start by trying to understand how to read and look at the xdmf surface data
Then we can adjust the InSAR data as needed and see if we can leverage existing workflows for .grd data 

In [62]:
xdmfFilename = '/Users/hyin/agsd/projects/insar/2021_haiti/dynamic-rupture/data_tmp/dynamic-rupture/jobid_3422948/output_extracted-surface.xdmf'

#     """read unstructured free surface output and associated data compute cell_barycenter"""
# Returns: lons, lats, lonlat_barycenter, connect, U, V, W
lons, lats, lonlat_barycenter, connect, U, V, W = read_seissol_surface_data(xdmfFilename)

# connect is a (32526, 3) np array, where there are 35k trianges, and each column is the index of one vertex
# lons is a (16347,) np array, where there are 16k vertices

# Fvsm2 = interp.LinearNDInterpolator(lonlat_barycenter, U)
# ui2 = Fvsm2.__call__(locData)
# Fvsm2 = interp.LinearNDInterpolator(lonlat_barycenter, V)
# vi2 = Fvsm2.__call__(locData)
# Fvsm2 = interp.LinearNDInterpolator(lonlat_barycenter, W)
# wi2 = Fvsm2.__call__(locData)

# ui2, vi2, wi2 should be the components of surface deformation at observations points (locData)


In [63]:
# I don't understand what read_seissol_surface_data(xdmfFilename) is doing so I'm going to run it line by line here: 

xdmfFilename = '/Users/hyin/agsd/projects/insar/2021_haiti/dynamic-rupture/data_tmp/dynamic-rupture/jobid_3422948/output_extracted-surface.xdmf'


sx = seissolxdmf.seissolxdmf(xdmfFilename)

# x, y, z components of each point where the surface displacement is defined. 
xyz = sx.ReadGeometry() # xyz.shape is (16347, 3) np arrray
connect = sx.ReadConnect()  # connect.shape is (32526, 3) np array

# u1, u2, u3 are surface deformations at point
# @todo: look up the sx.ReadData and sx.ReadConnect, sx.ndt class operators 
# (maybe in the seissoldxdmf pypi page)
U = sx.ReadData("u1", sx.ndt - 1)   
V = sx.ReadData("u2", sx.ndt - 1)
W = sx.ReadData("u3", sx.ndt - 1)

# project the data to geocentric coords (lat, lon)
# @todo: need to check this to adapt for my case 
# In Haiti, UTM is "epgs:32618"
myproj = "+proj=tmerc +datum=WGS84 +k=0.9996 +lon_0=37.0 +lat_0=37.0"
# myproj = "epgs:32618"
transformer = Transformer.from_crs(myproj, "epsg:4326", always_xy=True) # WGS84
lons, lats = transformer.transform(xyz[:, 0], xyz[:, 1])
xy = np.vstack((lons, lats)).T  # Creates a grid of lon,lat values, need to check the format

# compute triangule barycenter
# lonlat_barycenter is a 32526 x 2 array, so there should be 32526 triangles
lonlat_barycenter = (
    xy[connect[:, 0], :] + xy[connect[:, 1], :] + xy[connect[:, 2], :]
) / 3.0

lons.shape


(16347,)

## Read in Observation data

In [60]:
band='los184'       # define the datatype
downsampling=1      # Downsampling factor

if band in ["los184"]:
    # Plot InSAR track 184
    # Open geotif file
    src=rasterio.open('/Users/hyin/ags_local/data/haiti_2021_insar/ALOS2_A042/20210101_20210827/los_ll_dtr.tif')
    obs_to_plot=src.read(1)
    obs_to_plot[obs_to_plot==-32767]=np.nan
    obs_to_plot =  obs_to_plot/1e3  # Convert from mm to m

    # # Read in observations @mathilde: are vx, vy, and vz the component of the look vector??
    # fn = "/home/mathilde/Bureau/Work/Turquie/data/zeyu/scanSAR_new/184/vx_ll_low.tif"
    # lon_g, lat_g, vx = read_observation_data_one_band(fn)
    # fn = "/home/mathilde/Bureau/Work/Turquie/data/zeyu/scanSAR_new/184/vy_ll_low.tif"
    # lon_g, lat_g, vy = read_observation_data_one_band(fn)
    # fn = "/home/mathilde/Bureau/Work/Turquie/data/zeyu/scanSAR_new/184/vz_ll_low.tif"
    # lon_g, lat_g, vz = read_observation_data_one_band(fn)
elif band in ["los77"]:
    # Plot InSAR track 77
    src=rasterio.open('/home/mathilde/Bureau/Work/Turquie/data/zeyu/scanSAR_new/77/los_ll_low_nodata_filled.tif')
    obs_to_plot=src.read(1)
    obs_to_plot[obs_to_plot==-32767]=np.nan
    obs_to_plot =  obs_to_plot/1e2
    fn = "/home/mathilde/Bureau/Work/Turquie/data/zeyu/scanSAR_new/77/vx_ll_low.tif"
    lon_g, lat_g, vx = read_observation_data_one_band(fn)
    fn = "/home/mathilde/Bureau/Work/Turquie/data/zeyu/scanSAR_new/77/vy_ll_low.tif"
    lon_g, lat_g, vy = read_observation_data_one_band(fn)
    fn = "/home/mathilde/Bureau/Work/Turquie/data/zeyu/scanSAR_new/77/vz_ll_low.tif"
    lon_g, lat_g, vz = read_observation_data_one_band(fn)    
elif band in ["azimuth", "range"]:
    fn = f"/home/mathilde/Bureau/Work/Turquie/data/sar_offset/Displacement_TUR_20230114_20230207_1529_Data/20230114_HH_20230207_HH.spo_{band}.filtered.geo.tif"
    lon_g, lat_g, obsLOS = read_observation_data_one_band(fn)
    src=rasterio.open(f"/home/mathilde/Bureau/Work/Turquie/data/sar_offset/Displacement_TUR_20230114_20230207_1529_Data/20230114_HH_20230207_HH.spo_{band}.filtered.geo.tif")
    obs_to_plot=src.read(1)
    obs_to_plot[obs_to_plot==0]=np.nan
    fn = "/home/mathilde/Bureau/Work/Turquie/data/sar_offset/Displacement_TUR_20230114_20230207_1529_Data/20230114_HH_lv_phi.geo.tif"
    lon_g, lat_g, phi_g = read_observation_data_one_band(fn)
    fn = "/home/mathilde/Bureau/Work/Turquie/data/sar_offset/Displacement_TUR_20230114_20230207_1529_Data/20230114_HH_lv_theta.geo.tif"
    lon_g, lat_g, theta_g = read_observation_data_one_band(fn)

# src
obs_to_plot

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32)

## Interpolate SeisSol ouput at data locations

In [52]:
xdmfFilename2 = '/Users/hyin/agsd/projects/insar/2021_haiti/dynamic-rupture/data_tmp/dynamic-rupture/jobid_3422948/output_extracted-surface.xdmf'

## Read in XDMF file
lons, lats, lonlat_barycenter, connect, U, V, W=read_seissol_surface_data(xdmfFilename2)

# # Read SeisSol output and interpolate output to GPS data point locations
# xdmffile = '/Users/hyin/agsd/projects/insar/2021_haiti/dynamic-rupture/data_tmp/dynamic-rupture/jobid_3422948/output_extracted-surface.xdmf'

# lons, lats, lonlat_barycenter, connect, U, V, W = read_seissol_surface_data(
#     xdmffile, event=1
# )

ui, vi, wi = interpolate_seissol_surf_output(lonlat_barycenter, U, df)


In [56]:
## Load InSAR Data
filebase = 'A2_A043_20210818-20210901_los'
df = pd.read_csv('data/csv/'+filebase+'.csv')

# Write to geodataframe, with lon / lat fields as geometry
# Set geometry reference frame as WGS84
gdf = gpd.GeoDataFrame(
    geometry=gpd.points_from_xy(df.lon, df.lat, df.elevation, crs="EPSG:4326"), data=df    
)

# Convert from WGS to UTM
gdf_utm = gdf.to_crs(epsg=32618)

# Pull UTM coords 
coords_df = gdf_utm.get_coordinates(include_z=True)
coords_df

NameError: name 'pd' is not defined

In [53]:
# Get the lat lon location of observation data

# Fvsm = interp.LinearNDInterpolator(lonlat_barycenter, U)

## I don't understand what format it expects observation data/locations in
# Need to figure out what format locData should be in .
# Pull observation lats and lons


coords_df = gdf_utm.get_coordinates(include_z=True)


# lon_g = 
# lat_g = 

# locData = np.vstack((lon_g.flatten(),lat_g.flatten())).T


AttributeError: 'list' object has no attribute 'flatten'

In [49]:


# ui1 = Fvsm.__call__(locData)
# Fvsm = interp.LinearNDInterpolator(lonlat_barycenter, V)
# vi1 = Fvsm.__call__(locData)
# Fvsm = interp.LinearNDInterpolator(lonlat_barycenter, W)
# wi1 = Fvsm.__call__(locData)    


# I think this duplicate step was intended to sum synthetic contributions from 
# both events in the Turkey doublet if needed. Can ignore for now. 
lons2, lats2, lonlat_barycenter2, connect2, U2, V2, W2=read_seissol_surface_data(xdmfFilename2)

# ui, vi, wi should be  the interpolated values of deformation at observation points
Fvsm2 = interp.LinearNDInterpolator(lonlat_barycenter2, U2)
ui2 = Fvsm2.__call__(locData)
Fvsm2 = interp.LinearNDInterpolator(lonlat_barycenter2, V2)
vi2 = Fvsm2.__call__(locData)
Fvsm2 = interp.LinearNDInterpolator(lonlat_barycenter2, W2)
wi2 = Fvsm2.__call__(locData)

#ui = ui1 + ui2
#vi = vi1 + vi2
#wi = wi1 + wi2

ui = ui2
vi = vi2
wi = wi2

ui

FileNotFoundError: [Errno 2] No such file or directory: '/home/mathilde/Bureau/Work/Turquie/dynamic_rupture/prefered_model_thomas/ouput_model_2603_s205/Turkey_2events_31mio_o5_2603_s205last-surface.xdmf'

In [ ]:

# Plot figure
vmax = 4.0
vmin = -vmax
if band in ["azimuth", "range"]:
    extentmap=[36.6, 38.4, 36.7, 38.2]
else:
    extentmap=[36, 38.8, 36.0, 38.6]

fig = plt.figure()
fig.set_size_inches(5, 4.5)
ax = []
ax.append(fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree()))
setup_map(ax[0],extentmap)
show(obs_to_plot, transform=src.transform, cmap=cm.vik,vmin=vmin,vmax=vmax)

if band=='los77':
    syn_to_plot = compute_LOS_displacement_SeisSol_data2(
        lon_g, lat_g, vx.flatten(), vy.flatten(), vz.flatten(), lonlat_barycenter, ui,vi,wi)
    syn_to_plot = np.reshape(syn_to_plot, (np.max(np.shape(vx)), np.min(np.shape(vx))))
elif band=='los184':
    syn_to_plot = compute_LOS_displacement_SeisSol_data2(
        lon_g, lat_g, vx.flatten(), vy.flatten(), vz.flatten(), lonlat_barycenter, ui,vi,wi)
    syn_to_plot = np.reshape(syn_to_plot, (np.max(np.shape(vx)), np.min(np.shape(vx))))    
elif band=='EW':
    syn_to_plot = ui
    syn_to_plot = np.reshape(syn_to_plot, (np.min(np.shape(obs_to_plot)), np.max(np.shape(obs_to_plot))))
elif band=='NS':
    syn_to_plot = vi
    syn_to_plot = np.reshape(syn_to_plot, (np.min(np.shape(obs_to_plot)), np.max(np.shape(obs_to_plot))))    
elif band in ['azimuth','range']:
    syn_to_plot = compute_LOS_displacement_SeisSol_data(
                lon_g, lat_g, theta_g.flatten(), phi_g.flatten(), lonlat_barycenter, ui, vi, wi)
    syn_to_plot = np.reshape(syn_to_plot, (np.max(np.shape(obs_to_plot)), np.min(np.shape(obs_to_plot))))        

# Remove predictions where data is NaN
ind=np.isnan(obs_to_plot)
syn_to_plot[ind]=np.nan

## Plot residuals
#ind=np.isfinite(obs_to_plot)
res=obs_to_plot - syn_to_plot
ind=np.isfinite(res)
res = res[ind]

N=max(np.shape(res))
rms=np.sqrt(np.nansum(res**2)/N);
rms2 = nanrms(res)
print('RMS',band,':',np.round(rms,4),'m')


if band in ['azimuth','range']:
    ax.append(fig.add_axes([.485, 0.13, 0.40,0.40],projection=ccrs.PlateCarree()))
else:
    ax.append(fig.add_axes([.485, 0.12, 0.40,0.40],projection=ccrs.PlateCarree()))
    #ax.append(fig.add_axes([.485, 0.12, 0.40,0.40],projection=ccrs.PlateCarree()))  
    
ax[1].set_extent(extentmap)
setup_map(ax[1],extentmap,gridlines_left=False, draw_labels=False)
show(syn_to_plot, transform=src.transform, cmap=cm.vik,vmin=-vmax,vmax=vmax) 

#fn = f"comparison_geodetic_{band}_TSR.png"
#plt.savefig(fn, dpi=300, bbox_inches="tight")
#print(f"done writing {fn}")
#plt.show()